# MCP

## Summary - MCP Server Primitives
- Tools - Model controlled - LLM decides when to call them; results used by LLM; Used - additional functionality to LLM
- Resources - App-controlled - our App decides when to call these; used by our app; Used - getting data into app, adding context to messages
- User-controlled - the User decides when to use them; Used - workflows to run based on user input (e.g. slash command/button click/menu option)

## Introduction
- MCP - communication protocol - Sharing Tools, Resources, Prompts...etc that are very specialised (and not efficient to perform via 'local' tools by each local/Server) -> MCP Servers (remote) - exposed to external services. No need to author functions, schemas, prompts...etc. Also - no need to create functions that call external services/APIs.
- Multiple MCP Servers available to client
- MCP Servers - can be made by anyone - Service Providers - create official MCP Server implementations (tools schemas/functions).
- MCP Server - Tools, Prompts, Resources (& Links to outside services)
- Resources - determine behaviour/configuration of MCP server/tools (routing, interacting with DB...etc). Define server-side properties and behaviors in your MCP application. They are located in a resources directory in your project

## MCP Client
- MCP Client - Communication between our Server and MCP Server.
- MCP - transport agnostic - they can communicate over many protocols
- Locally/same machine - Standard IO (MCP Server implemented locally)
- Remote - HTTP, Websockets...etc
- Client needs to access - Tools, Resources, Prompts in MCP Server
- MCP communication (spec) specifies different types of messages that can be exchanged
 - Main message types: ListToolsRequest, ListToolsResult, CallToolRequest, CallToolResult, ReadResourceRequest, ReadResourceResults

- Example - Full Message Flow: User - Our 'app' Server - MCP Client - MCP Server - Claude - Ext Service (Github)
1. User - ask our Server - 'what repos do i have'
2. Our 'app' Server - ask MCP Client - List of Tools to send to Claude alongside user prompt
3. MCP Client - send ListToolsRequest to MCP Server
4. MCP Server - respond ListToolsResult to MCP Client
5. Our 'app' Server - send user Query + Tools list to Claude
6. Claude decides user tool - send ToolUse request to Server
7. Server sends request to MCP Client with specific arguments
8. MCP Client sends CallToolRequest to MCP Server
9. MCP Server sends Request to External Service (e.g. Github)
10. Ext Service (Github) response to MCP Server
11. MCP Server sends CallToolResult to MCP Client
12. MCP Client sends result of running tool to our 'app' Server
13. Our Server has toolResult (full list of repos) and sends to Claude
14. Claude provides full answer using toolResult to our 'app' Server
15. 'App' Server provides final result back to user

# Hands-on Project

## UV FOR VENV/PACKAGE MANAGEMENT
1. Create requirements.txt or pyproject.toml file
2. Create Venv with UV


```
# pyproject.toml
[project]
name = "app"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.10"
dependencies = [
    "anthropic>=0.51.0",
    "mcp[cli]>=1.8.0",
    "prompt-toolkit>=3.0.51",
    "python-dotenv>=1.1.0",
]
```

```
 uv venv (within project)
 source .venv/bin/activate

```

## MCP Server Inspector
- MCP Server Inspector - Python MCP SDK built-in browser - test functionality without connecting to full application. Debug and test server in real-time.
- Dev deployment (similar to npm deployment), any changes in server are DEV and not persisted (ephemeral)
```
uv venv
mcp dev mcp_server.py
Proxy server listening on http:///xxx
```





## CLI-based chatbot (interact with documents)
- MCP Client (inside our Server)
- MCP Server - 2 tools (1-Read doc, 2-Update doc)
- Documents (external in MCP, but in this project - in memory; fake 'external/3rd party')
- NOTE - normally you implement either MCP Client OR Server, not both (doesn't make sense). But here we're doing both to understand how it works.
-

- We start with CLI project (main.py) which works well - it's a CLI based chatbot that talks to Claude.
- - MCP Client Class - wrapper of Session Class (actual connection to MCP server)
- Allows us to get tools and resources from MCP Server

- The CLI App has an MCP Server and MCP Client (skeletons). We'll start by building out the MCP Server, adding tools.
- MCP SDK - great abstraction to build Servers & Clients in different languages
- Python MCP SDK - create MCP server very easy and Create tools very easy (Decorators). No need to create Complex JSON Schemas.
```
@mcp.tool_name()
def_function()
```



## MCP Server - Resources
- Resources - Resources provide structured access to information that the AI application can retrieve and provide to models as context. Resources expose data from files, APIs, databases, or any other source that an AI needs to understand context. Applications can access this information directly and decide how to use it - whether that’s selecting relevant portions, searching with embeddings, or passing it all to the model.
- They allow MCP server to expose data to the client; can return any type of data (strings/JSON/binary...etc); we set mime_type to give client a hint of data we're returning
- 2 types: Direct (static URI) and Templated (templated URI - parameters; matches param in resource function)
- In our Project, we're implementing @Document_name feature - this will enable 1)the CLI auto-complete (i.e. showing a list of available documents) and 2)Claude additional context (via system prompt) to understand @ is referencing a document name directly.
- Use ReadResourceRequest, ReadResourceResults calls
- Use URI (address of the resource)
- Resources need to be accessible to MCP Client so it can parse/return to Server(user)
- Resources expose data which in some cases mean Claude don't need to make Tool Calls
```
@mcp.resource(
    "docs://documents",
    mime_type='application/json'
)
def list_docs():
    #Returns list of document names - then this will show in cli (through CLI class)

@mcp.resource(
    "docs://documents/{doc_id}",
    mime_type='application/json'
)
def fetch_doc(doc_id: str):
    #Returns content - then this will show in cli (through CLI class)

```

## MCP Server - Prompts
- Set of User and Assistant messages - can be used by client
- Thoroughly validated, well-tested, high quality prompts, relevant to overall MCP purpose - improves responses from LLM and better outputs for users.
- Prompt parameters - passed to prompt function (must be passed from Client function/call & match)
```
@mcp.tool_name(
    name="..",
    description="..."
)
def_function(
    ...

) -> list[base.Message]:
    prompt = ...
```